In [1]:
#load python packages
import os
import pandas as pd

In [2]:
path = os.getcwd()
file_name = 'fake_job_postings.csv'
path += "/../data/raw/"+file_name

In [3]:
df = pd.read_csv(path, index_col='job_id')

# Dataset description
### String

| Column Name  | Description                                    |
|--------------|------------------------------------------------|
| Title        | The title of the job ad entry.                 |
| Location     | Geographical location of the job ad.           |
| Department   | Corporate department (e.g. sales).             |
| Salary range | Indicative salary range (e.g. $50,000-$60,000) |

### HTML fragment

| Column Name     | Description                               |
|-----------------|-------------------------------------------|
| Company profile | A brief company description               |
| Description     | The details description of the job ad     |
| Requirements    | Enlisted requirements for the job opening |
| Benefits        | Enlisted offered benefits by the employer |

### Binary

| Column Name   | Description                              |
|---------------|------------------------------------------|
| Telecommuting | True for telecommuting positions.        |
| Company logo  | True if company logo is present.         |
| Questions     | True if screening questions are present. |
| Fraudulent    | Classification attribute.                |

### Nominal

| Column Name         | Description                                    |
|---------------------|------------------------------------------------|
| Employment type     | Full-type, Part-time, Contract, etc.           |
| Required experience | Executive, Entry level, Intern, etc.           |
| Required education  | Doctorate, Master's Degree, Bachelor, etc.     |
| Industry            | Automotive, IT, Health care, Real estate, etc. |
| Function            | Consulting, Engineering, Research, Sales, etc. |

We have a mix of text data, binary, numerical. Some of the text is categorical,
although it looks like I'll have to do some condensing of similar categories e.g. art manager -> manager

In [4]:
df.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
job_id,,,,,,,,,,,,,,,,,
1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [5]:
df.columns

Index(['title', 'location', 'department', 'salary_range', 'company_profile',
       'description', 'requirements', 'benefits', 'telecommuting',
       'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

Some observations: Most values for department and salary_range are null. Almost half of benefits,
required_experience, and required_education are null. It may be helpful to create additional
columns with null/not null for these

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17880 entries, 1 to 17880
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                17880 non-null  object
 1   location             17534 non-null  object
 2   department           6333 non-null   object
 3   salary_range         2868 non-null   object
 4   company_profile      14572 non-null  object
 5   description          17879 non-null  object
 6   requirements         15185 non-null  object
 7   benefits             10670 non-null  object
 8   telecommuting        17880 non-null  int64 
 9   has_company_logo     17880 non-null  int64 
 10  has_questions        17880 non-null  int64 
 11  employment_type      14409 non-null  object
 12  required_experience  10830 non-null  object
 13  required_education   9775 non-null   object
 14  industry             12977 non-null  object
 15  function             11425 non-null  object
 16  frau

In [7]:
# The percentage of unique values in each column
(df.nunique()/df.shape[0])*100

title                  62.813199
location               17.365772
department              7.477629
salary_range            4.888143
company_profile         9.558166
description            82.779642
requirements           66.935123
benefits               34.703579
telecommuting           0.011186
has_company_logo        0.011186
has_questions           0.011186
employment_type         0.027964
required_experience     0.039150
required_education      0.072707
industry                0.732662
function                0.206935
fraudulent              0.011186
dtype: float64

Note that only 5% of the data is fraudulent. We will need to fix the imbalance, probably with oversampling or a metric that values the less represented class.

In [8]:
df.describe()

,telecommuting,has_company_logo,has_questions,fraudulent
count,17880.000000,17880.000000,17880.000000,17880.000000
mean,0.042897,0.795302,0.491723,0.048434
std,0.202631,0.403492,0.499945,0.214688
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [9]:
#check for duplicate rows, looks like we have 281 of them
duplicateRowsDF = df[df.duplicated()]
duplicateRowsDF.shape

(281, 17)

In [10]:
df.drop_duplicates(inplace=True)

The most represented industries are all tech. To keep the data manageable I may need to only take
the top most represented industries and relabel the others as 'Other' 

In [11]:
pd.set_option('display.max_rows', 150)
df.industry.value_counts()

Information Technology and Services     1711
Computer Software                       1362
Internet                                1054
Education Management                     819
Marketing and Advertising                818
Financial Services                       748
Hospital & Health Care                   483
Consumer Services                        348
Telecommunications                       326
Oil & Energy                             286
Retail                                   223
Real Estate                              166
Accounting                               159
Construction                             152
E-Learning                               138
Design                                   127
Staffing and Recruiting                  127
Management Consulting                    127
Health, Wellness and Fitness             123
Insurance                                121
Automotive                               117
Logistics and Supply Chain               110
Human Reso

In [12]:
df.title.nunique()

11231

I'm planning on using NLP techniques to reduce the titles by combining similar ones.

In [13]:
df.title.value_counts()

English Teacher Abroad                                309
Graduates: English Teacher Abroad (Conversational)    144
Customer Service Associate                            141
English Teacher Abroad                                 94
Software Engineer                                      86
                                                     ... 
Hilshire Farms Senior Strategic Pricing Analyst         1
Intake Coordinator                                      1
Javascript Front-end Engineer                           1
Business Analyst                                        1
Web Developer (Marketing)                               1
Name: title, Length: 11231, dtype: int64

Now let's remove NA values. First check how many in each column

In [14]:
df.isnull().sum()

title                      0
location                 341
department             11328
salary_range           14772
company_profile         3282
description                1
requirements            2648
benefits                7106
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3428
required_experience     6962
required_education      8009
industry                4837
function                6363
fraudulent                 0
dtype: int64

I decided to modify these columns into empty (0) or not (1), to simplify my modeling while not throwing out the columns entirely: department, salary_range, benefits, required_experience, required_education

In [15]:
temp_df = df[['department', 'salary_range', 'benefits', 'required_experience', 'required_education']]
temp_df[~temp_df.isnull()] = 1
temp_df[temp_df.isnull()] = 0
temp_df = temp_df.add_prefix('has_')
df.drop(['department', 'salary_range', 'benefits', 'required_experience', 'required_education'], axis=1, inplace=True)
df = df.join(temp_df)

/home/joe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/joe/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
/home/joe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

I would like to split location into separate columns- country, state, city

In [16]:
#the first few rows seem to be structred as Country, State, City. I'd like to confirm the whole column conforms to that
loc_df = df.location
test = loc_df.str.split(',').to_frame('location')
test['length']=test.location.str.len()
test.sort_values("length", ascending=False).head()
#looks like I need to cut it off at the third comma and remove everything after that

,location,length
job_id,,
13182,"[US, CA, Los Angeles, California, San Fran...",16.0
17572,"[US, FL, Orlando, Lake City, Jacksonville, ...",11.0
4651,"[US, FL, Central Fl, Nort Fl, Georgia, Al...",9.0
12468,"[US, NJ, Lakewood, Brick, Toms River, Jacks...",7.0
4528,"[US, NJ, Lakewood, Brick, Toms River, Jacks...",7.0


In [17]:
loc_df = loc_df.str.split(',', expand=True)[[0,1,2]]
loc_df.columns = ['Country','State','City']
loc_df.head()
#Perfect!

,Country,State,City
job_id,,,
1,US,NY,New York
2,NZ,,Auckland
3,US,IA,Wever
4,US,DC,Washington
5,US,FL,Fort Worth


In [18]:
#add those columns back in replacing the original location column
df.drop('location',axis=1,inplace=True)
df = df.join(loc_df)
df.head()

,title,company_profile,description,requirements,telecommuting,has_company_logo,has_questions,employment_type,industry,function,fraudulent,has_department,has_salary_range,has_benefits,has_required_experience,has_required_education,Country,State,City
job_id,,,,,,,,,,,,,,,,,,,
1,Marketing Intern,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0,1,0,Other,NaN,Marketing,0,1,0,0,1,0,US,NY,New York
2,Customer Service - Cloud Video Production,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0,1,0,Full-time,Marketing and Advertising,Customer Service,0,1,0,1,1,0,NZ,,Auckland
3,Commissioning Machinery Assistant (CMA),Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0,1,0,NaN,NaN,NaN,0,0,0,0,0,0,US,IA,Wever
4,Account Executive - Washington DC,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0,1,0,Full-time,Computer Software,Sales,0,1,0,1,1,1,US,DC,Washington
5,Bill Review Manager,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0,1,1,Full-time,Hospital & Health Care,Health Care Provider,0,0,0,1,1,1,US,FL,Fort Worth


I will simplify the Country values into a few of the most represented categories and dummify them

Here's some functions I'll use to clean up the rest of the columns

In [19]:
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
from collections import Counter

def remove_punctuation(text):
    no_punct=[letter for letter in text if letter not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

def tokenize(text):
    #note this splits up contractions into separate whole words, and gives punctuation like comma its own token
    return nltk.word_tokenize(text)

def remove_stopwords(text):
    #applies to a list of words, so need to tokenize before using this
    stopword = nltk.corpus.stopwords.words('english')
    text=[word for word in text if word not in stopword]
    return text

def remove_numbers(text):
    text_nonum = re.sub(r'\d+', '', text)
    return text_nonum

def remove_uppercase(text):
    return text.lower()

def process_text(text, lemmatize=False):
    processed_text = remove_stopwords(tokenize(remove_numbers(remove_uppercase(remove_punctuation(text)))))
    if lemmatize:
        L = WordNetLemmatizer()
        processed_text = [L.lemmatize(word) for word in processed_text]
    return processed_text

def most_common_words(df_column, display_n=50):
    '''
    input: df_column- single column of pandas dataframe, display_n- amount of words to display
    displays the n most common words found in df_column
    '''
    processed_column = df_column.apply(process_text)
    word_list = []
    for row in processed_column:
        word_list.extend(row)
    c=Counter(word_list)
    return c.most_common(display_n)

def keep_words(lst, keep_list):
    return [word for word in lst if word in keep_list]

def condense_categories(df_column, keep_list):
    processed_column = df_column.apply(process_text)
    return processed_column.apply(lambda x:' '.join((keep_words(x, keep_list))))

def get_top_dummify(df_column, top_n):
    '''
    takes in a categorical column and an integer top_n
    returns the top n most represented categories in df_column, dummified
    '''
    to_keep = list(df_column.value_counts().iloc[:top_n-1].index) #grab top n most represented categories
    to_keep = [cat for cat in to_keep if cat] #filter out empty categories
    new_column = df_column.where(df_column.isin(to_keep), 'Other')
    return new_column.str.get_dummies().add_prefix(df_column.name+str('_'))

def split_top_categories(df,col_name,top_n):
    dummy_df = get_top_dummify(df[col_name], top_n)
    df = df.drop(col_name, axis=1)
    return df.join(dummy_df)
    

[nltk_data] Downloading package stopwords to /home/joe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/joe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
df.Country.value_counts().head(10)

US    10460
GB     2329
GR      938
CA      450
DE      382
NZ      330
IN      273
AU      213
PH      132
NL      126
Name: Country, dtype: int64

In [21]:
df = split_top_categories(df,'Country', 5)

In [22]:
df.State.value_counts().head(10)

        2108
 CA     2012
 NY     1228
 LND     991
 TX      957
 I       686
 IL      411
 FL      410
 OH      365
 VA      324
Name: State, dtype: int64

In [23]:
df = split_top_categories(df,'State', 5)

In [24]:
df.City.value_counts().head(10)

                  1619
 London           1052
 New York          655
 Athens            541
 San Francisco     472
 Houston           267
 Berlin            262
 Washington        257
 Chicago           252
 Auckland          226
Name: City, dtype: int64

In [25]:
df = split_top_categories(df,'City', 5)

Next, I'd like to simplify and condense some of the categorical columns into a few of the top categories.
These columns will be affected: title, employment_type, industry, function

In [26]:
'''
Title column first. Too many categories here. I'd like to find common words like 'teacher', 'engineer',
'manager' and split title into those groups.
'''
df.title.value_counts()

English Teacher Abroad                                309
Graduates: English Teacher Abroad (Conversational)    144
Customer Service Associate                            141
English Teacher Abroad                                 94
Software Engineer                                      86
                                                     ... 
Hilshire Farms Senior Strategic Pricing Analyst         1
Intake Coordinator                                      1
Javascript Front-end Engineer                           1
Business Analyst                                        1
Web Developer (Marketing)                               1
Name: title, Length: 11231, dtype: int64

In [27]:
'''
I'm going to somewhat arbitrarily cut this off at "consultant", which would be the first 32 words.
Any entry that does not contain these words will have all 0s in the final dummified columns
'''
title_word_counts = most_common_words(df.title, 35)
print(title_word_counts)

[('manager', 2195), ('developer', 1780), ('engineer', 1592), ('sales', 1284), ('senior', 966), ('customer', 936), ('service', 859), ('english', 777), ('teacher', 772), ('marketing', 748), ('abroad', 689), ('assistant', 645), ('associate', 639), ('software', 639), ('designer', 585), ('web', 531), ('business', 510), ('specialist', 501), ('account', 488), ('executive', 483), ('representative', 473), ('director', 463), ('analyst', 461), ('development', 385), ('lead', 371), ('support', 354), ('product', 348), ('data', 344), ('project', 340), ('intern', 287), ('consultant', 285), ('time', 265), ('operations', 258), ('technical', 255), ('year', 250)]


In [28]:
#replacing title column simplified and dummified titles
title_words_keep = [word for word,count in title_word_counts[0:31]]
title_dummies = condense_categories(df.title,title_words_keep).str.get_dummies(sep=' ').add_prefix('title_')
df = df.join(title_dummies)
df.drop('title',axis=1, inplace=True)
df.head()

,company_profile,description,requirements,telecommuting,has_company_logo,has_questions,employment_type,industry,function,fraudulent,...,title_project,title_representative,title_sales,title_senior,title_service,title_software,title_specialist,title_support,title_teacher,title_web
job_id,,,,,,,,,,,,,,,,,,,,,
1,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0,1,0,Other,NaN,Marketing,0,...,0,0,0,0,0,0,0,0,0,0
2,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0,1,0,Full-time,Marketing and Advertising,Customer Service,0,...,0,0,0,0,1,0,0,0,0,0
3,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0,1,0,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
4,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0,1,0,Full-time,Computer Software,Sales,0,...,0,0,0,0,0,0,0,0,0,0
5,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0,1,1,Full-time,Hospital & Health Care,Health Care Provider,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
'''
employment_type next. Looks like I don't need to simplify other than fixing NAs.
I'll dummify this column and null values will get all 0's in the dummified columns
'''
print(df.employment_type.value_counts())
print('count of na values: '+str(df.employment_type.isnull().sum()))

Full-time    11423
Contract      1514
Part-time      772
Temporary      237
Other          225
Name: employment_type, dtype: int64
count of na values: 3428


In [30]:
'''
employment_dummies = pd.get_dummies(df.employment_type, prefix='employment_type_')
df.drop('employment_type', axis=1, inplace=True)
df = df.join(employment_dummies)
'''
df = split_top_categories(df,'employment_type',5)
df.head()

,company_profile,description,requirements,telecommuting,has_company_logo,has_questions,industry,function,fraudulent,has_department,...,title_software,title_specialist,title_support,title_teacher,title_web,employment_type_Contract,employment_type_Full-time,employment_type_Other,employment_type_Part-time,employment_type_Temporary
job_id,,,,,,,,,,,,,,,,,,,,,
1,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0,1,0,NaN,Marketing,0,1,...,0,0,0,0,0,0,0,1,0,0
2,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0,1,0,Marketing and Advertising,Customer Service,0,1,...,0,0,0,0,0,0,1,0,0,0
3,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0,1,0,NaN,NaN,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0,1,0,Computer Software,Sales,0,1,...,0,0,0,0,0,0,1,0,0,0
5,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0,1,1,Hospital & Health Care,Health Care Provider,0,0,...,0,0,0,0,0,0,1,0,0,0


In [31]:
'''
industry next. These look good, but too many small categories. I'm going to arbitrarily
cut it off at the 10 most numerous categories
'''
df.industry.value_counts()

Information Technology and Services     1711
Computer Software                       1362
Internet                                1054
Education Management                     819
Marketing and Advertising                818
Financial Services                       748
Hospital & Health Care                   483
Consumer Services                        348
Telecommunications                       326
Oil & Energy                             286
Retail                                   223
Real Estate                              166
Accounting                               159
Construction                             152
E-Learning                               138
Design                                   127
Staffing and Recruiting                  127
Management Consulting                    127
Health, Wellness and Fitness             123
Insurance                                121
Automotive                               117
Logistics and Supply Chain               110
Human Reso

In [32]:
'''industry_keep = list(df.industry.value_counts().iloc[:9].index)
industry_df = df.industry.where(df.industry.isin(industry_keep), 'Other')
df = df.join(industry_df.str.get_dummies().add_prefix('industry_'))
df.drop('industry', axis=1, inplace=True)
'''
df = split_top_categories(df,'industry',10)
df.head()

,company_profile,description,requirements,telecommuting,has_company_logo,has_questions,function,fraudulent,has_department,has_salary_range,...,industry_Computer Software,industry_Consumer Services,industry_Education Management,industry_Financial Services,industry_Hospital & Health Care,industry_Information Technology and Services,industry_Internet,industry_Marketing and Advertising,industry_Other,industry_Telecommunications
job_id,,,,,,,,,,,,,,,,,,,,,
1,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0,1,0,Marketing,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0,1,0,Customer Service,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0,1,0,NaN,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0,1,0,Sales,0,1,0,...,1,0,0,0,0,0,0,0,0,0
5,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0,1,1,Health Care Provider,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [33]:
'''
function next. This looks similar to industry, so same solution- arbitrarily pick top 10 and
the rest become 'other'
'''
df.function.value_counts().head(20)

Information Technology    1730
Sales                     1448
Engineering               1339
Customer Service          1180
Marketing                  818
Administrative             613
Design                     336
Health Care Provider       326
Education                  325
Other                      325
Management                 306
Business Development       226
Accounting/Auditing        210
Human Resources            201
Project Management         183
Finance                    165
Consulting                 138
Writing/Editing            131
Art/Creative               131
Production                 115
Name: function, dtype: int64

In [34]:
df = split_top_categories(df,'function',10)

df.head()

,company_profile,description,requirements,telecommuting,has_company_logo,has_questions,fraudulent,has_department,has_salary_range,has_benefits,...,function_Administrative,function_Customer Service,function_Design,function_Education,function_Engineering,function_Health Care Provider,function_Information Technology,function_Marketing,function_Other,function_Sales
job_id,,,,,,,,,,,,,,,,,,,,,
1,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0,1,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0
3,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
5,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0,1,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


Next, I'd like to apply the usual NLP preprocessing techniques to my text columns: remove non-letter characters, tokenization, lowercase all characters, remove stop words, lemmatization
This will affect all remaining text columns: company_profile, description, requirements

In [35]:
#first fill NAs in the final three columns
df = df.fillna('None')


text_columns = ['company_profile', 'description', 'requirements']
for col in text_columns:
    unprocessed_col = df[col]
    processed_col = unprocessed_col.apply(process_text, lemmatize=True)
    df.drop(col, axis=1, inplace=True)
    df = df.join(processed_col)

Finally, I'll add three more features: the length in words of my three text columns: company_profile, description, requirements. These will be my only numeric columns until I use an embedding on my text columns later on.

In [36]:
for col in text_columns:
    col_lengths = df[col].str.len()
    df = df.join(col_lengths, rsuffix='_length')

In [38]:
df.head()

,telecommuting,has_company_logo,has_questions,fraudulent,has_department,has_salary_range,has_benefits,has_required_experience,has_required_education,Country_CA,...,function_Information Technology,function_Marketing,function_Other,function_Sales,company_profile,description,requirements,company_profile_length,description_length,requirements_length
job_id,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,1,0,0,1,0,0,...,0,1,0,0,"[food, weve, created, groundbreaking, awardwin...","[food, fastgrowing, james, beard, awardwinning...","[experience, content, management, system, majo...",88,79,73
2,0,1,0,0,1,0,1,1,0,0,...,0,0,0,0,"[second, world, cloud, video, production, serv...","[organised, focused, vibrant, awesomedo, passi...","[expect, youyour, key, responsibility, communi...",99,192,119
3,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,"[valor, service, provides, workforce, solution...","[client, located, houston, actively, seeking, ...","[implement, precommissioning, commissioning, p...",82,30,107
4,0,1,0,0,1,0,1,1,1,0,...,0,0,0,1,"[passion, improving, quality, life, geography,...","[company, esri, –, environmental, system, rese...","[education, bachelor, ’, master, ’, gi, busine...",57,226,124
5,0,1,1,0,0,0,1,1,1,0,...,0,0,0,0,"[spotsource, solution, llc, global, human, cap...","[job, title, itemization, review, managerlocat...","[qualificationsrn, license, state, texasdiplom...",151,126,62


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17599 entries, 1 to 17880
Data columns (total 86 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   telecommuting                                 17599 non-null  int64 
 1   has_company_logo                              17599 non-null  int64 
 2   has_questions                                 17599 non-null  int64 
 3   fraudulent                                    17599 non-null  int64 
 4   has_department                                17599 non-null  int64 
 5   has_salary_range                              17599 non-null  int64 
 6   has_benefits                                  17599 non-null  int64 
 7   has_required_experience                       17599 non-null  int64 
 8   has_required_education                        17599 non-null  int64 
 9   Country_CA                                    17599 non-null  int64 
 10

All columns except my three text features have been properly dummified. The text columns have been preprocessed and are ready for NLP models/transformations (like embedding). There are no NA values remaining. Done with wrangling!

In [40]:
df.to_csv('../data/interim/fake_job_postings_preprocessed.csv')